In [18]:
import os
import sys
import numpy as np
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from importlib import reload
import math
from IPython.display import display, Markdown
from datetime import datetime

sys.path.insert(0, os.path.abspath('../tesi_sabella'))
# pd.options.display.max_rows = 999

import pyfluxc.pyfluxc as flux
import data_generator as generator
reload(flux)
reload(generator)

<module 'data_generator' from '/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/tesi_sabella/data_generator.py'>

In [20]:
# Constants ----- #
BUCKET = 'CIC_IDS_2017_Monday'
PULL_EVERY_X_MINUTES = 15
PULL_DELTA_TIME = pd.Timedelta(minutes=PULL_EVERY_X_MINUTES)

In [19]:
# Functions ----- #
def mprint(s):
    display(Markdown(s))

# Generating time series

We pull only the time range in which samples are available

In [7]:
fclient = flux.FluxClient(); 
start, end = fclient.bucket_timerange(BUCKET)
print(f"first sample: {start}\nlast sample: {end}\nTotal: {end-start}")

first sample: 2020-04-04 17:18:30
last sample: 2020-04-05 08:48:55
Total: 0 days 15:30:25


In [30]:
cicids2017 = generator.CICIDS2017(BUCKET, '15s', fclient, start)

Extract data from influxdb in time intervals due to influx kill signal

In [ ]:
i = 1
pull_start = start
while True:
    pull_end = pull_start + PULL_DELTA_TIME
    
    print("------------")
    print(f"first sample: {pull_start}\nlast sample: {pull_end}\nTotal: {pull_end-pull_start}")
    print("------------")
    
    # TODO: return tuple
    pull_start, new_samples = cicids2017.pull(stop=pull_end)
    if new_samples is None:
        break
    
cicids2017_df = cicids2017.to_pandas()

Generic information

In [25]:
cicids2017_df.to_pickle(f'./{BUCKET}.pkl')